In [ ]:
from google.colab import drive
import os
import ee
import geemap
import pandas as pd

# Mount Google Drive
drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/GEE_Exports')
!pwd

cloud_project = 'ee-1wangjas2'

try:
    ee.Initialize(project=cloud_project)
except:
    ee.Authenticate()
    ee.Initialize(project=cloud_project)

# Set variables
country = "Canada"
province = "Alberta"
year = "2022"
dataset_name = "MODIS/061/MCD12Q1"
start_date = f'{year}-01-01'
end_date = f'{year}-12-31'
epsm = 1000  # resolution in meters
attributes = ['LC_Type1', 'LW']  # List of attributes
file_name = "landcover_zonal_yearly_with_lw.csv"

# Load MODIS Land Cover dataset
modis_landcover = ee.ImageCollection(dataset_name)

# Filter the dataset for the range and select necessary bands
modis_filtered = modis_landcover.filterDate(start_date, end_date).select(attributes)
print("Filtered ImageCollection size (number of images):", modis_filtered.size().getInfo())

# Load level 2 administrative boundaries from GAUL
gaul_cities = ee.FeatureCollection('FAO/GAUL/2015/level2')

# Filter for Canada and the selected province
canada_cities = gaul_cities.filter(ee.Filter.eq('ADM0_NAME', country))
province_cities = canada_cities.filter(ee.Filter.eq('ADM1_NAME', province))

# Use province geometry as the region
region_geometry = province_cities.geometry()

# Create a grid over the selected region
projection = ee.Projection('EPSG:4326').atScale(epsm)
grids = region_geometry.coveringGrid(projection)

# Compute zonal stats for each grid cell for each image
def compute_zonal_stats_per_grid(image):
    def process_grid(grid):
        stats = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=grid.geometry(),
            scale=epsm,
            maxPixels=1e13,
            bestEffort=True
        )
        # Get grid cell centroid
        centroid = grid.geometry().centroid(maxError=1)
        stats = stats.set('longitude', centroid.coordinates().get(0)) \
                     .set('latitude', centroid.coordinates().get(1))
        return ee.Feature(None, stats)

    # Map over each grid in the region
    return grids.map(process_grid)

# Apply the function to all images in the collection
zonal_stats = modis_filtered.map(compute_zonal_stats_per_grid)

# Flatten the collection of collections into a single FeatureCollection
zonal_stats_fc = ee.FeatureCollection(zonal_stats.flatten())

# Export the dataset as a CSV with grid-level details
export_task = ee.batch.Export.table.toDrive(
    collection=zonal_stats_fc,
    description=f'{province}_Landcover',
    folder='GEE_Exports',
    fileFormat='CSV',
    selectors=['longitude', 'latitude'] + attributes
)

# Start the export task
export_task.start()
print("Export task started. Check Earth Engine Code Editor for progress.")


Mounted at /content/drive
/content/drive/My Drive/GEE_Exports
Filtered ImageCollection size (number of images): 1
Export task started. Check Earth Engine Code Editor for progress.
